# Music21 Scale Tables

In this notebook we are developing a eye-candy presentation of scale data generated by music21 and our music21 toolchain.

## Web of Reference

### Music / Sheet Music

* [music21](http://web.mit.edu/music21/doc/moduleReference/index.html)
* [abcjs](https://github.com/paulrosen/abcjs)

### Tables

* [plim](https://plim.readthedocs.io/en/latest/syntax.html)
* [pandas]() / [style](http://pandas.pydata.org/pandas-docs/version/0.18.1/style.html)
* [d3.js](https://d3js.org/)

### Colors

* matplotlib
  * [Pyplot colormap line by line – StackOverflow](http://stackoverflow.com/questions/27443050/pyplot-colormap-line-by-line)
  * [Choosing Colormaps – Matplotlib](http://matplotlib.org/users/colormaps.html)
  * [cm (colormap)](http://matplotlib.org/api/cm_api.html)
  * [colors – Matplotlib](http://matplotlib.org/api/colors_api.html)
* [colormap (pypi)](https://pypi.python.org/pypi/colormap)
* [palettable](https://jiffyclub.github.io/palettable) / [source](https://github.com/jiffyclub/palettable)
* [pallete (pypi)](https://pypi.python.org/pypi/palette)
* [colour (pypi)](https://pypi.python.org/pypi/colour)
* [Solarized Color Scheme](http://ethanschoonover.com/solarized)
* [Seaborn](http://chrisalbon.com/python/seaborn_color_palettes.html)
* [Colour Science](http://colour-science.org/)

### Audio

* [Tone.js](https://github.com/Tonejs/Tone.js)
* [MIDI.js](https://github.com/mudcube/MIDI.js)

### Persistency

* sqlite3
* [sqlalchemy](http://docs.sqlalchemy.org/en/rel_1_1/)

In [23]:
from music21 import *
from IPython.display import display, HTML, Image, Audio
import plim
import base64
#from music21.common.pathTools import relativepath as m21_jupyter_relpath

In [24]:
def circleOfFifths():
    fifths = []

    fifthsScale = scale.CyclicalScale('c4', 'p5')
    
    for p in fifthsScale.getPitches('c4', 'c11'):
        
        if key.pitchToSharps(p,'major') <= 7:
            MAJOR = str(p.simplifyEnharmonic().name)
            MINOR = str(key.Key(MAJOR, 'major').relative.tonic.simplifyEnharmonic().name).lower()

        else:
            MAJOR = str(p.getEnharmonic().simplifyEnharmonic().name)
            MINOR = str(key.Key(MAJOR, 'major').relative.tonic.simplifyEnharmonic().name).lower()
        
        fifths.append( [ MAJOR, MINOR ] )

        #fifths.append(p.simplifyEnharmonic())
        
    return fifths

## The returnScale() function

This function returns a `dict()` with the data and filename of our generated scale.

In [25]:
def returnScale(tonic='c', mode='major', ownKey = False):
    
    modes = [ 'ionian','dorian','phrygian',
              'lydian','mixolydian','aeolian',
              'locrian','major','minor']
    
    if not mode in modes:
        mode = 'major'
        
    if mode == 'ionian': mode = 'major'
    if mode == 'aeolian': mode = 'minor'

    tonic_pitch = pitch.Pitch(tonic)
    #key_enharmonic = tonic_pitch.getEnharmonic().simplifyEnharmonic().name
    key_enharmonic = tonic_pitch
    scale_sharps = key.pitchToSharps(key_enharmonic, mode)

    #    myfunc = eval("scale.{0}Scale".format(mode.capitalize()))
    scale_func = getattr(scale, "{0}Scale".format(mode.capitalize()))
    real_scale = scale_func(key_enharmonic)
    pitches = real_scale.getPitches()

    scale_name = real_scale.name
    absolute_intervals = []
    relative_intervals = []
    myKeySig = key.KeySignature(scale_sharps)
    
    mystream = stream.Stream()
    
    if ownKey:
        mystream.append(myKeySig)

    for x in range(0, len(pitches)):
        mynote = note.Note(pitches[x])

        absolute_interval = interval.notesToInterval(pitches[0], mynote).directedName
        relative_interval = interval.notesToInterval(pitches[(x-1 if x>0 else 0)], mynote).directedName
        
        absolute_intervals.append(absolute_interval)
        relative_intervals.append(relative_interval)
        
        mynote.addLyric(absolute_interval)
        mynote.addLyric(relative_interval)
        
        mystream.append(mynote)
        
    filename = mystream.write()
        
    return {
        'scalename': scale_name,
        'stream' : mystream,
        'filename': filename,
        'ai' : absolute_intervals,
        'ri' : relative_intervals
    }

## Creating an Array with scale data

Here we create a bidimensional array with our scale data—a list of lists.

In [40]:
#my_data = returnScale(scale_data)
def generate_table_array(scale_data):
    
    # lets embed the image
    with open(scale_data['filename'],'rb') as image_file:
        b64_image_data = base64.b64encode(image_file.read()).decode('utf8')

    imgtag  = "<img src='data:image/png;charset=utf-8;base64,{}' />".format(b64_image_data)

    my_table_data = [
        [ scale_data['scalename'] ],
        [ imgtag ],
        scale_data['ai'], # these are already lists
        scale_data['ri']
        ]
    return my_table_data

## Converting the data array to HTML

Here we use python `plim` library to help us convert the data array to a HTML table, the which we can then present on the Jupyter Notebook.

In [45]:
def array_to_htmltable(data):
    t2_slim = """
table style='float: left;'
"""

    for row in data:
        t2_slim += "\ttr\n"
        for column in row:
            if len(row) == 1:
                t2_slim += "\t\tcaption {}\n".format(column)
            else:
                t2_slim += "\t\ttd {}\n".format(column)

    return plim.preprocessor(t2_slim)

## Displaying the table

Then we can show the IPython.display to show the HTML table.

In [37]:
HTML(table_html)

P1,M2,M3,P4,P5,M6,M7,P8
P1,M2,M2,m2,M2,M2,M2,m2


In [30]:
Fifths = circleOfFifths()

Fifths

[['C', 'a'],
 ['G', 'e'],
 ['D', 'b'],
 ['A', 'f#'],
 ['E', 'c#'],
 ['B', 'g#'],
 ['F#', 'd#'],
 ['C#', 'a#'],
 ['A-', 'f'],
 ['E-', 'c'],
 ['B-', 'g'],
 ['F', 'd'],
 ['C', 'a']]

In [47]:
for major, minor in Fifths:
    scale_data = returnScale(major,ownKey=True)
    table_array = generate_table_array(scale_data)
    html_table = array_to_htmltable(table_array)
    display(HTML(html_table))

P1,M2,M3,P4,P5,M6,M7,P8
P1,M2,M2,m2,M2,M2,M2,m2


P1,M2,M3,P4,P5,M6,M7,P8
P1,M2,M2,m2,M2,M2,M2,m2


P1,M2,M3,P4,P5,M6,M7,P8
P1,M2,M2,m2,M2,M2,M2,m2


P1,M2,M3,P4,P5,M6,M7,P8
P1,M2,M2,m2,M2,M2,M2,m2


P1,M2,M3,P4,P5,M6,M7,P8
P1,M2,M2,m2,M2,M2,M2,m2


...

P1,M2,M3,P4,P5,M6,M7,P8
P1,M2,M2,m2,M2,M2,M2,m2


P1,M2,M3,P4,P5,M6,M7,P8
P1,M2,M2,m2,M2,M2,M2,m2


P1,M2,M3,P4,P5,M6,M7,P8
P1,M2,M2,m2,M2,M2,M2,m2


P1,M2,M3,P4,P5,M6,M7,P8
P1,M2,M2,m2,M2,M2,M2,m2


P1,M2,M3,P4,P5,M6,M7,P8
P1,M2,M2,m2,M2,M2,M2,m2
